In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
questions = pd.DataFrame.from_csv('question_simple.csv', index_col=None)
questions.head()

,QuestionId,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,QScore,QVotes
0,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,12,12
1,2,2011-11-15T20:50:11.270,2011-12-02T11:38:48.620,10,2011-11-16T00:00:00.000,12,12
2,3,2011-11-15T20:52:27.287,2015-07-18T09:49:54.953,109,2013-01-19T00:00:00.000,26,30
3,7,2011-11-15T20:56:40.383,2015-11-23T12:25:29.887,2977,2012-10-22T00:00:00.000,23,26
4,14,2011-11-15T21:01:05.117,2015-08-13T13:14:56.870,20,2011-11-16T00:00:00.000,11,12


In [3]:
a_questions = pd.DataFrame.from_csv('question_votes.csv', index_col=None)
a_questions[a_questions.AcceptedAnsId.notnull()].head()

,QuestionId,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,VoteType,QVoteCreation
0,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
1,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
2,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
3,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037
4,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2,2011-11-15T20:45:51.037


In [4]:
get_votes_qv = lambda df: pd.Series((df.VoteType==2).cumsum() + (df.VoteType==3).cumsum(),name='QVotes')
get_score_qv = lambda df: pd.Series((df.VoteType==2).cumsum() - (df.VoteType==3).cumsum(),name='QScore')

predictors_qvotes = ['QuestionId','QuestionCreation','QuestionLastActivity','AcceptedAnsId','AcceptedDate','QVoteCreation']
f_q = lambda df: pd.concat([df[cname] for cname in df.columns.values.tolist() if cname in predictors_qvotes]+[get_score_qv(df),get_votes_qv(df)],axis=1)
a_questions = a_questions.sort_values(by='QVoteCreation').groupby(['QuestionId']).apply(f_q)
a_questions.head(10)

,QuestionId,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,QVoteCreation,QScore,QVotes
0,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,1,1
11,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,2,2
10,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,3,3
9,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,4,4
7,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,5,5
6,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,6,6
8,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,7,7
4,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,8,8
3,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,9,9
2,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,10,10


In [5]:
a_votes = pd.DataFrame.from_csv('votes-answers.csv', index_col=None)
a_votes = pd.merge(a_votes, a_questions, how='inner', on=['QuestionId'],suffixes=['_v', '_q'])
a_votes

,VoteId,VoteCreation,AnsCreation,VoteType,AnsId,QuestionId,AnsWordCount,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate,QVoteCreation,QScore,QVotes
0,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,1,1
1,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,2,2
2,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,3,3
3,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,4,4
4,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,5,5
5,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,6,6
6,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,7,7
7,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,8,8
8,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,9,9
9,2,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000,2011-11-15T20:45:51.037,10,10


In [6]:
predictors_raw_votans =['VoteId','VoteCreation','AnsCreation','VoteType','AnsId','QuestionId','AnsWordCount','QuestionCreation','AcceptedAnsId','AcceptedDate']

valid_qavotes = lambda df: df[df.VoteCreation>=df.QVoteCreation]
#Use twice valid_qavotes, could use once to improve efficiency, but check correctness of index selection
get_max_qv = lambda df: valid_qavotes(df).loc[valid_qavotes(df).QVotes.idxmax(),['QScore','QVotes']].squeeze()
get_latest_qv = lambda df : pd.Series([0,0],index=['QScore','QVotes']) if not (df.VoteCreation>=df.QVoteCreation).any() else get_max_qv(df)
get_head = lambda df: [df[cname].iloc[0] for cname in df.columns.values.tolist() if cname in predictors_raw_votans]
get_qv = lambda df : pd.Series(get_head(df),index=predictors_raw_votans).append(get_latest_qv(df)).to_frame()

a_votes = a_votes.sort_values(by='VoteCreation').groupby(['VoteId']).apply(get_qv).unstack(level=-1).reset_index(level=[0],drop=True)
a_votes.drop(a_votes.columns[[0]], axis=1, inplace=True)
a_votes.columns = a_votes.columns.droplevel()
a_votes

,VoteCreation,AnsCreation,VoteType,AnsId,QuestionId,AnsWordCount,QuestionCreation,AcceptedAnsId,AcceptedDate,QScore,QVotes
0,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
1,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
2,2011-11-15T00:00:00.000,2011-11-15T20:54:09.083,2,5,1,37,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
3,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,3,4,1,190,2011-11-15T20:45:51.037,56,2011-11-21T00:00:00.000,0,0
4,2011-11-15T00:00:00.000,2011-11-15T20:54:49.410,2,6,2,51,2011-11-15T20:50:11.270,10,2011-11-16T00:00:00.000,0,0
5,2011-11-15T00:00:00.000,2011-11-15T20:58:23.697,2,10,2,145,2011-11-15T20:50:11.270,10,2011-11-16T00:00:00.000,0,0
6,2011-11-15T00:00:00.000,2011-11-15T21:00:06.700,2,13,2,89,2011-11-15T20:50:11.270,10,2011-11-16T00:00:00.000,0,0
7,2011-11-15T00:00:00.000,2011-11-15T20:57:41.503,2,8,2,46,2011-11-15T20:50:11.270,10,2011-11-16T00:00:00.000,0,0
8,2011-11-15T00:00:00.000,2011-11-15T21:00:06.700,2,13,2,89,2011-11-15T20:50:11.270,10,2011-11-16T00:00:00.000,0,0
9,2011-11-15T00:00:00.000,2011-11-15T20:58:23.697,2,10,2,145,2011-11-15T20:50:11.270,10,2011-11-16T00:00:00.000,0,0


In [7]:
a_votes['AcceptedDate'].fillna(pd.to_datetime('20160320'),inplace=True) #Date After Data Set Collection
a_votes['AcceptedAge'] = (pd.to_datetime(a_votes.AcceptedDate,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['AcceptedAge'] = a_votes['AcceptedAge'] + 1

a_votes.loc[a_votes.AcceptedDate == pd.to_datetime('20160320'), 'AcceptedAge'] = -1

a_votes['RelAge'] = (pd.to_datetime(a_votes.VoteCreation,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.AnsCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['RelAge'] = a_votes['RelAge'] + 1

a_votes['Age'] = (pd.to_datetime(a_votes.VoteCreation,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['Age'] = a_votes['Age'] + 1
a_votes.drop(a_votes.columns[[0, 1, 6, 8]], axis=1, inplace=True)
a_votes

,VoteType,AnsId,QuestionId,AnsWordCount,AcceptedAnsId,QScore,QVotes,AcceptedAge,RelAge,Age
0,2,4,1,190,56,0,0,6,0,0
1,2,4,1,190,56,0,0,6,0,0
2,2,5,1,37,56,0,0,6,0,0
3,3,4,1,190,56,0,0,6,0,0
4,2,6,2,51,10,0,0,1,0,0
5,2,10,2,145,10,0,0,1,0,0
6,2,13,2,89,10,0,0,1,0,0
7,2,8,2,46,10,0,0,1,0,0
8,2,13,2,89,10,0,0,1,0,0
9,2,10,2,145,10,0,0,1,0,0


In [8]:
a_comments = pd.DataFrame.from_csv('comment-ans.csv', index_col=None)
a_comments = pd.merge(a_comments, questions, how='inner', on=['QuestionId'],suffixes=['_vot', '_quest'])

a_comments['Age'] = (pd.to_datetime(a_comments.CommentCreation,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_comments.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_comments['Age'] = a_comments['Age'] + 1
a_comments = a_comments[['AnsId','Age']]
a_comments.head(10)

,AnsId,Age
0,4,1
1,4,1
2,5,1
3,56,1
4,56,1
5,56,1
6,56,1
7,7030,951
8,7030,1285
9,15104,1436


## Each entry in the following tables represents the total votes in an answer in a given day (and associated attributes for the answer and question)

In [9]:
get_score = lambda df: sum(df.VoteType==2) - sum(df.VoteType==3)
get_votes = lambda df: sum(df.VoteType==2) + sum(df.VoteType==3)

predictors = ['QuestionId','AnsWordCount','AcceptedAnsId','AcceptedAge','QScore','QVotes','Score','Votes']
f = lambda df: pd.Series([df.QuestionId.iloc[0],df.AnsWordCount.iloc[0],df.AcceptedAnsId.iloc[0],df.AcceptedAge.iloc[0],df.QScore.iloc[0],df.QVotes.iloc[0],get_score(df),get_votes(df)],index = predictors)
a_groups = a_votes.sort_values(by='Age').groupby(['AnsId','Age']).apply(f)
a_groups = a_groups.reset_index(level=[0,1],drop=False)
a_groups

,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes
0,4,0,1,190,56,6,0,0,1,3
1,4,1436,1,190,56,6,12,12,1,1
2,5,0,1,37,56,6,0,0,1,1
3,5,108,1,37,56,6,12,12,-1,1
4,6,0,2,51,10,1,0,0,1,1
5,8,0,2,46,10,1,0,0,1,1
6,9,0,3,25,109,431,0,0,4,4
7,9,12,3,25,109,431,26,26,1,1
8,9,361,3,25,109,431,26,26,1,1
9,9,400,3,25,109,431,26,26,1,1


In [10]:
cum_votes = lambda df: pd.Series(df['Votes'].cumsum(),name='CumVotes')
cum_score = lambda df: pd.Series(df['Score'].cumsum(),name='CumScore')

f =lambda df: pd.concat([df[cname] for cname in df.columns.values.tolist()] + [cum_votes(df),cum_score(df)],axis=1)
a_groups_c = a_groups.sort_values(by='Age').groupby(['AnsId']).apply(f)
a_groups_c

,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,CumVotes,CumScore
0,4,0,1,190,56,6,0,0,1,3,3,1
9873,5380,0,5379,43,5380,0,0,0,1,1,1,1
9872,5378,0,5377,51,5378,0,0,0,2,2,2,2
9868,5375,0,5373,241,NaN,-1,0,0,1,1,1,1
2475,858,0,857,91,858,39,0,0,3,3,3,3
9857,5368,0,5367,235,5368,0,0,0,1,1,1,1
9852,5363,0,5360,389,5363,3,0,0,1,1,1,1
9851,5362,0,5360,44,5363,3,0,0,1,1,1,1
9849,5359,0,5357,136,5359,0,0,0,2,2,2,2
9844,5358,0,5350,428,NaN,-1,0,0,2,2,2,2


In [11]:
a_groups_c['ReScore'] = a_groups_c['CumScore']/(a_groups_c['CumVotes']+1.0)
a_groups_c['QReScore'] = a_groups_c['QScore']/(a_groups_c['QVotes']+1.0)
#votes['TScore'] = np.power(votes['Score'],votes['age_vot'])
a_groups_c

,AnsId,Age,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,CumVotes,CumScore,ReScore,QReScore
0,4,0,1,190,56,6,0,0,1,3,3,1,0.250000,0.000000
9873,5380,0,5379,43,5380,0,0,0,1,1,1,1,0.500000,0.000000
9872,5378,0,5377,51,5378,0,0,0,2,2,2,2,0.666667,0.000000
9868,5375,0,5373,241,NaN,-1,0,0,1,1,1,1,0.500000,0.000000
2475,858,0,857,91,858,39,0,0,3,3,3,3,0.750000,0.000000
9857,5368,0,5367,235,5368,0,0,0,1,1,1,1,0.500000,0.000000
9852,5363,0,5360,389,5363,3,0,0,1,1,1,1,0.500000,0.000000
9851,5362,0,5360,44,5363,3,0,0,1,1,1,1,0.500000,0.000000
9849,5359,0,5357,136,5359,0,0,0,2,2,2,2,0.666667,0.000000
9844,5358,0,5350,428,NaN,-1,0,0,2,2,2,2,0.666667,0.000000


In [12]:
votes_com = pd.merge(a_groups_c, a_comments, how='left', on=['AnsId'],suffixes=['_vot', '_com'])
f = lambda df:pd.concat([df[cname] for cname in df.columns.values.tolist()]+[pd.Series(df['Age_vot']>=df['Age_com'],name='gt')],axis=1)
votes_com = votes_com.groupby(['AnsId']).apply(f)
votes_com.head(10)

,AnsId,Age_vot,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,CumVotes,CumScore,ReScore,QReScore,Age_com,gt
0,4,0,1,190,56,6,0,0,1,3,3,1,0.250000,0,1,False
1,4,0,1,190,56,6,0,0,1,3,3,1,0.250000,0,1,False
2,5380,0,5379,43,5380,0,0,0,1,1,1,1,0.500000,0,1,False
3,5380,0,5379,43,5380,0,0,0,1,1,1,1,0.500000,0,1,False
4,5380,0,5379,43,5380,0,0,0,1,1,1,1,0.500000,0,1,False
5,5380,0,5379,43,5380,0,0,0,1,1,1,1,0.500000,0,1,False
6,5378,0,5377,51,5378,0,0,0,2,2,2,2,0.666667,0,NaN,False
7,5375,0,5373,241,NaN,-1,0,0,1,1,1,1,0.500000,0,1,False
8,5375,0,5373,241,NaN,-1,0,0,1,1,1,1,0.500000,0,2,False
9,5375,0,5373,241,NaN,-1,0,0,1,1,1,1,0.500000,0,2,False


In [13]:
tmp_sum_com = votes_com.groupby(['AnsId','Age_vot']).apply(lambda df: sum(df['gt']))
tmp_sum_com.reset_index(drop=False).head(10)

,AnsId,Age_vot,0
0,4,0,0
1,4,1436,2
2,5,0,0
3,5,108,1
4,6,0,0
5,8,0,0
6,9,0,0
7,9,12,3
8,9,361,3
9,9,400,3


## First Time series cleaned (model for the number of votes)

In [14]:
votes_com_f = pd.merge(votes_com,tmp_sum_com.reset_index(drop=False),on=['AnsId','Age_vot'],how='inner')
votes_com_f.drop(votes_com_f.columns[[15]], axis=1, inplace=True)
votes_com_f.columns.values[15]='Comments'
votes_com_f = votes_com_f.drop_duplicates()
votes_com_f

,AnsId,Age_vot,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,CumVotes,CumScore,ReScore,QReScore,Age_com,Comments
0,4,0,1,190,56,6,0,0,1,3,3,1,0.250000,0.000000,1,0
2,5380,0,5379,43,5380,0,0,0,1,1,1,1,0.500000,0.000000,1,0
6,5378,0,5377,51,5378,0,0,0,2,2,2,2,0.666667,0.000000,NaN,0
7,5375,0,5373,241,NaN,-1,0,0,1,1,1,1,0.500000,0.000000,1,0
8,5375,0,5373,241,NaN,-1,0,0,1,1,1,1,0.500000,0.000000,2,0
10,858,0,857,91,858,39,0,0,3,3,3,3,0.750000,0.000000,1,0
15,858,0,857,91,858,39,0,0,3,3,3,3,0.750000,0.000000,4,0
17,5368,0,5367,235,5368,0,0,0,1,1,1,1,0.500000,0.000000,1,0
18,5363,0,5360,389,5363,3,0,0,1,1,1,1,0.500000,0.000000,1,0
19,5362,0,5360,44,5363,3,0,0,1,1,1,1,0.500000,0.000000,NaN,0


# Check inconsistent entries

In [15]:
votes_com_f[(votes_com_f['AnsId'].isnull()) | (votes_com_f['Age_vot'].isnull()) | (votes_com_f['Votes'].isnull()) | (votes_com_f['Score'].isnull()) | (votes_com_f['Comments'].isnull())]

,AnsId,Age_vot,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,CumVotes,CumScore,ReScore,QReScore,Age_com,Comments


In [16]:
votes_com_f[votes_com_f.Age_vot < 0]

,AnsId,Age_vot,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,CumVotes,CumScore,ReScore,QReScore,Age_com,Comments


In [17]:
votes_com_f[votes_com_f.CumScore>votes_com_f.CumVotes]

,AnsId,Age_vot,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,CumVotes,CumScore,ReScore,QReScore,Age_com,Comments


In [18]:
#Accepted answers 687 and 990 didn't have acceptance dates registered?
votes_com_f[(votes_com_f['AcceptedAnsId'].notnull()) & (votes_com_f['AcceptedAge']<0)]

,AnsId,Age_vot,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,QScore,QVotes,Score,Votes,CumVotes,CumScore,ReScore,QReScore,Age_com,Comments
886,990,0,987,62,990,-1,0,0,1,1,1,1,0.500000,0.000000,1,0
889,990,0,987,62,990,-1,0,0,1,1,1,1,0.500000,0.000000,64,0
890,990,0,987,62,990,-1,0,0,1,1,1,1,0.500000,0.000000,70,0
891,990,0,987,62,990,-1,0,0,1,1,1,1,0.500000,0.000000,76,0
1824,687,0,686,31,687,-1,0,0,5,5,5,5,0.833333,0.000000,1,0
1830,688,0,686,26,687,-1,0,0,4,4,4,4,0.800000,0.000000,1,0
1833,689,0,686,48,687,-1,0,0,3,3,3,3,0.750000,0.000000,1,0
1837,690,0,686,39,687,-1,0,0,1,1,1,1,0.500000,0.000000,NaN,0
16312,687,4,686,31,687,-1,4,4,1,1,6,6,0.857143,0.800000,1,5
23422,687,35,686,31,687,-1,4,4,1,1,7,7,0.875000,0.800000,1,5


## Calculate Rank

In [19]:
from itertools import izip
def rank_ans(df,score_only,re_score):
    rk_name = "ReScore_rank" if re_score else "AnsRank"
    def rank_iter():
        cache = {}
        accepted = 0
        for row in df.itertuples():
            if re_score:
                cache[row.AnsId] = row.ReScore
            else :
                cache[row.AnsId] = row.Score
            # rank, nb_ans
            if (not score_only) and row.AcceptedAge>-1 and (row.AnsId == row.AcceptedAnsId) and row.Age_vot >=row.AcceptedAge:
                accepted = 1
                if row.AnsId in cache:
                    del cache[row.AnsId]
                yield (1,len(cache)+accepted,row.Index)
            else :
                rank = sorted(cache, key= lambda k:cache[k],reverse=True).index(row.AnsId) + 1 + accepted
                yield (rank,len(cache)+accepted,row.Index)
            
    ranks, ans_counts, indices = izip(*list(rank_iter())) #TODO: optimize for the future
    return [pd.Series(ranks,name=rk_name, index=indices), pd.Series(ans_counts,name="Ans_count", index=indices)]

predictors = ['AnsId','Comments','AnsWordCount','AcceptedAnsId','Age_vot','Score','Votes','CumScore','CumVotes','QScore','QVotes','ReScore','QReScore','AnsRank','ReScore_rank']
get_ranks = lambda df,score_only=False,re_score=False: pd.concat([df[cname] for cname in df.columns.values.tolist() if cname in predictors] + rank_ans(df,score_only,re_score),axis=1)
sort_age_score = lambda df: df.sort_values(by=['Age_vot','Score'],ascending=[True,False])

## Base data for model

In [20]:
votes_com_f = votes_com_f.groupby(['QuestionId']).apply(lambda df: get_ranks(sort_age_score(df)))
votes_com_f = votes_com_f.reset_index(level=[0],drop=False)
votes_com_f = votes_com_f.groupby(['QuestionId']).apply(lambda df: get_ranks(sort_age_score(df),score_only=True,re_score=True))
votes_com_f = votes_com_f.reset_index(level=[0],drop=False)

votes_com_f['Pbias'] = 1.0/votes_com_f['AnsRank']
votes_com_f['DRank'] = votes_com_f['AnsRank'] - votes_com_f['ReScore_rank']
votes_com_f.head(20)

,QuestionId,AnsId,Age_vot,AnsWordCount,AcceptedAnsId,QScore,QVotes,Score,Votes,CumVotes,CumScore,ReScore,QReScore,Comments,AnsRank,ReScore_rank,Ans_count,Pbias,DRank
4484,1,56,0,201,56,0,0,2,2,2,2,0.666667,0.000000,0,1,1,1,1.000000,0
0,1,4,0,190,56,0,0,1,3,3,1,0.250000,0.000000,0,2,2,2,0.500000,0
5773,1,5,0,37,56,0,0,1,1,1,1,0.500000,0.000000,0,3,2,3,0.333333,1
10683,1,56,1,201,56,12,12,3,3,5,5,0.833333,0.923077,4,1,1,3,1.000000,0
10735,1,210,1,40,56,12,12,1,1,1,1,0.500000,0.923077,0,2,2,4,0.500000,0
16223,1,210,4,40,56,12,12,1,1,2,2,0.666667,0.923077,0,2,2,4,0.500000,0
16703,1,56,4,201,56,12,12,1,1,6,6,0.857143,0.923077,4,1,1,4,1.000000,0
18974,1,56,7,201,56,12,12,1,1,7,7,0.875000,0.923077,4,1,1,4,1.000000,0
25833,1,897,108,68,56,12,12,1,1,1,1,0.500000,0.923077,0,2,3,5,0.500000,-1
25834,1,5,108,37,56,12,12,-1,1,2,0,0.000000,0.923077,1,5,5,5,0.200000,0


In [21]:
sum_by_rank = lambda df: df.groupby('AnsRank').apply(lambda df: pd.Series([df.Votes.sum()],name='EPbias').to_frame()).unstack(level=-1).reset_index(level=0,drop=False)
get_ratio = lambda df: sum_by_rank(df).EPbias/(sum_by_rank(df).EPbias.sum())
ratio_per_rank = lambda df: pd.concat([sum_by_rank(df).AnsRank, get_ratio(df)],axis=1)
get_position_bias = lambda df: pd.merge(df,ratio_per_rank(df),how='inner',on=['AnsRank'])

votes = votes_com_f.groupby(['Ans_count']).apply(get_position_bias).reset_index(level=[0,1],drop=True)
votes.columns.values[-1] = "EPbias"
votes

,QuestionId,AnsId,Age_vot,AnsWordCount,AcceptedAnsId,QScore,QVotes,Score,Votes,CumVotes,CumScore,ReScore,QReScore,Comments,AnsRank,ReScore_rank,Ans_count,Pbias,DRank,EPbias
0,1,56,0,201,56,0,0,2,2,2,2,0.666667,0.000000,0,1,1,1,1.000000,0,1.000000
1,2,10,0,145,10,0,0,3,3,3,3,0.750000,0.000000,0,1,1,1,1.000000,0,1.000000
2,2,10,0,145,10,0,0,3,3,3,3,0.750000,0.000000,0,1,1,1,1.000000,0,1.000000
3,3,9,0,25,109,0,0,4,4,4,4,0.800000,0.000000,0,1,1,1,1.000000,0,1.000000
4,7,16,0,45,2977,0,0,3,3,3,3,0.750000,0.000000,0,1,1,1,1.000000,0,1.000000
5,14,20,0,28,20,0,0,7,7,7,7,0.875000,0.000000,0,1,1,1,1.000000,0,1.000000
6,18,33,0,133,33,0,0,3,3,3,3,0.750000,0.000000,0,1,1,1,1.000000,0,1.000000
7,18,33,0,133,33,0,0,3,3,3,3,0.750000,0.000000,0,1,1,1,1.000000,0,1.000000
8,18,33,0,133,33,0,0,3,3,3,3,0.750000,0.000000,0,1,1,1,1.000000,0,1.000000
9,21,80,0,43,80,0,0,1,1,1,1,0.500000,0.000000,0,1,1,1,1.000000,0,1.000000


In [22]:
votes.to_csv(path_or_buf='AnsVotes_TSeries.csv', header = [str(h) for h in votes.columns])

In [23]:
votes[['Age_vot','AnsWordCount','Comments','Ans_count']].describe()

,Age_vot,AnsWordCount,Comments,Ans_count
count,23328.000000,23328.000000,23328.000000,23328.000000
mean,212.595508,143.485039,2.033522,2.721622
std,380.532603,150.314528,2.851866,2.061537
min,0.000000,3.000000,0.000000,1.000000
25%,1.000000,51.000000,0.000000,1.000000
50%,7.000000,97.000000,1.000000,2.000000
75%,231.000000,178.000000,3.000000,3.000000
max,1504.000000,2142.000000,18.000000,12.000000


In [24]:
votes['Ans_count'].value_counts().head(15)

1     7132
2     6866
3     4033
4     2172
5     1169
6      535
10     484
7      283
8      273
9      240
11      73
12      68
Name: Ans_count, dtype: int64

In [25]:
#votes = votes[ votes.age_vot>=3]
votes = votes[votes.Age_vot<=7]
votes[['Score','Votes','CumScore','CumVotes','QScore','QVotes','ReScore','QReScore','Ans_count','DRank']].describe()

,Score,Votes,CumScore,CumVotes,QScore,QVotes,ReScore,QReScore,Ans_count,DRank
count,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000
mean,1.674785,1.787765,3.338020,3.505733,4.262030,4.482802,0.667576,0.523072,1.892125,0.069462
std,1.404004,1.326312,2.641695,2.575425,6.957248,7.212644,0.233654,0.380433,1.204971,0.589805
min,-3.000000,1.000000,-6.000000,1.000000,-7.000000,0.000000,-0.857143,-0.875000,1.000000,-4.000000
25%,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.500000,0.000000,1.000000,0.000000
50%,1.000000,1.000000,3.000000,3.000000,3.000000,3.000000,0.750000,0.666667,2.000000,0.000000
75%,2.000000,2.000000,5.000000,5.000000,6.000000,6.000000,0.833333,0.833333,2.000000,0.000000
max,12.000000,12.000000,18.000000,18.000000,58.000000,62.000000,0.947368,0.979592,9.000000,8.000000


In [26]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

# How does question quality relate to answer quality

In [46]:
one_ans = smf.ols(formula='QReScore ~ ReScore + C(AnsRank) - 1', data=votes[votes['Ans_count']==1])
re_one = one_ans.fit()
print re_one.summary()

                            OLS Regression Results                            
Dep. Variable:               QReScore   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     411.6
Date:                Sat, 02 Apr 2016   Prob (F-statistic):           2.27e-88
Time:                        00:55:50   Log-Likelihood:                -2537.2
No. Observations:                5617   AIC:                             5078.
Df Residuals:                    5615   BIC:                             5092.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
C(AnsRank)[1]    -0.0351      0.022     -1.598

In [119]:
two_ans = smf.ols(formula='QReScore ~ ReScore + C(AnsRank) ', data=votes[votes['Ans_count']==2])
re_two = two_ans.fit()
print re_two.summary()

                            OLS Regression Results                            
Dep. Variable:               QReScore   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     134.3
Date:                Fri, 01 Apr 2016   Prob (F-statistic):           4.27e-57
Time:                        21:30:52   Log-Likelihood:                -1250.8
No. Observations:                3847   AIC:                             2508.
Df Residuals:                    3844   BIC:                             2526.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           0.4914      0.018     

In [35]:
three_ans = smf.ols(formula='QReScore ~ ReScore + AnsWordCount + C(AnsRank) - 1', data=votes[votes['Ans_count']==3])
re_three = three_ans.fit()
print re_three.summary()

                            OLS Regression Results                            
Dep. Variable:               QReScore   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     48.03
Date:                Sat, 02 Apr 2016   Prob (F-statistic):           4.58e-38
Time:                        02:52:17   Log-Likelihood:                -241.94
No. Observations:                1516   AIC:                             493.9
Df Residuals:                    1511   BIC:                             520.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
C(AnsRank)[1]     0.4807      0.026     18.540

In [116]:
four_ans = smf.ols(formula='QReScore ~ ReScore + C(AnsRank) - 1', data=votes[votes['Ans_count']==4])
re_four = four_ans.fit()
print re_four.summary()

                            OLS Regression Results                            
Dep. Variable:               QReScore   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     13.62
Date:                Fri, 01 Apr 2016   Prob (F-statistic):           1.26e-10
Time:                        21:29:23   Log-Likelihood:                -94.607
No. Observations:                 593   AIC:                             199.2
Df Residuals:                     588   BIC:                             221.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
C(AnsRank)[1]     0.5687      0.036     15.955

In [117]:
five_ans = smf.ols(formula='QReScore ~ ReScore + C(AnsRank) - 1', data=votes[votes['Ans_count']==5])
re_five = five_ans.fit()
print re_five.summary()

                            OLS Regression Results                            
Dep. Variable:               QReScore   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     7.515
Date:                Fri, 01 Apr 2016   Prob (F-statistic):           2.07e-06
Time:                        21:29:27   Log-Likelihood:                 24.902
No. Observations:                 180   AIC:                            -37.80
Df Residuals:                     174   BIC:                            -18.65
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
C(AnsRank)[1]     0.5872      0.061      9.663

In [124]:
print "Pearson r Score-position: ", votes['Score'].corr(votes['AnsRank'],method='pearson')

Pearson r Score-position:  -0.261224567951


## Mixed effects model for votes

In [105]:
formula = "np.log(Votes+1) ~ C(AnsRank) - 1"
model = sm.MixedLM.from_formula(formula, votes, re_formula="AnsRank", groups=votes["AnsId"])
result = model.fit()
print result.summary()

c:\Users\Joel\Anaconda2\lib\site-packages\statsmodels\base\model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\Users\Joel\Anaconda2\lib\site-packages\statsmodels\regression\mixed_linear_model.py:1717: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   np.log(Votes + 1)
No. Observations:   12054     Method:               REML             
No. Groups:         4594      Scale:                0.0965           
Min. group size:    1         Likelihood:           -4144.6903       
Max. group size:    70        Converged:            Yes              
Mean group size:    2.6                                              
---------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
C(AnsRank)[1]              1.017    0.005 191.155 0.000  1.007  1.027
C(AnsRank)[2]              0.840    0.007 126.444 0.000  0.827  0.853
C(AnsRank)[3]              0.772    0.011  70.549 0.000  0.751  0.794
C(AnsRank)[4]              0.792    0.020  39.025 0.000  0.752  0.832
C(AnsRank)[5]              0.770    

In [106]:
formula = "np.log(Votes+1) ~ C(AnsRank,Helmert) - 1"
model = sm.MixedLM.from_formula(formula, votes, re_formula="AnsRank", groups=votes["AnsId"])
result = model.fit()
print result.summary()

                    Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     np.log(Votes + 1)
No. Observations:      12054       Method:                 REML             
No. Groups:            4594        Scale:                  0.0965           
Min. group size:       1           Likelihood:             -4157.5194       
Max. group size:       70          Converged:              Yes              
Mean group size:       2.6                                                  
----------------------------------------------------------------------------
                                 Coef.  Std.Err.    z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------
C(AnsRank, Helmert)[H.intercept]  0.893    0.045  19.996 0.000  0.805  0.981
C(AnsRank, Helmert)[H.2]         -0.089    0.004 -21.516 0.000 -0.097 -0.081
C(AnsRank, Helmert)[H.3]         -0.052    0.004 -13.306 0.000 -0.060 -0.044
C(AnsRank, Helmert

c:\Users\Joel\Anaconda2\lib\site-packages\statsmodels\regression\mixed_linear_model.py:1717: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [107]:
formula = "np.log(Votes+1) ~ C(AnsRank,Diff) - 1"
model = sm.MixedLM.from_formula(formula, votes, re_formula="AnsRank", groups=votes["AnsId"])
result = model.fit()
print result.summary()

c:\Users\Joel\Anaconda2\lib\site-packages\statsmodels\base\model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\Users\Joel\Anaconda2\lib\site-packages\statsmodels\regression\mixed_linear_model.py:1717: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   np.log(Votes + 1)
No. Observations:   12054     Method:               REML             
No. Groups:         4594      Scale:                0.0965           
Min. group size:    1         Likelihood:           -4144.7526       
Max. group size:    70        Converged:            Yes              
Mean group size:    2.6                                              
---------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
C(AnsRank, Diff)[D.1]      0.892    0.045  19.975 0.000  0.805  0.980
C(AnsRank, Diff)[D.2]     -0.177    0.008 -21.518 0.000 -0.194 -0.161
C(AnsRank, Diff)[D.3]     -0.067    0.013  -5.304 0.000 -0.092 -0.042
C(AnsRank, Diff)[D.4]      0.020    0.023   0.871 0.384 -0.025  0.065
C(AnsRank, Diff)[D.5]     -0.023    

## OLS, Poisson, RobustLM

In [33]:
#Comments Ans_count AnsWordCount DRank Age_vot 
mod_ols = smf.ols(formula='np.log(Votes+1) ~ ReScore -1', data=votes[votes.Age_vot<6])#&(votes.DRank==2)])
res = mod_ols.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:      np.log(Votes + 1)   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                 5.533e+04
Date:                Sat, 02 Apr 2016   Prob (F-statistic):               0.00
Time:                        02:25:02   Log-Likelihood:                -6188.6
No. Observations:               11112   AIC:                         1.238e+04
Df Residuals:                   11111   BIC:                         1.239e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
ReScore        1.3391      0.006    235.229      0.0

In [66]:
mod_poi = smf.glm('Votes ~ Score + Ans_rank + AnsWordCount + Comments - 1 ', data=votes,
                  family=sm.families.Poisson(link=sm.families.links.log)).fit(method='bfgs') #
#print mod_poi.mle_retvals['converged']
print mod_poi.summary()

NameError: name 'Ans_rank' is not defined

In [19]:
huber_t = sm.RLM(np.log(votes.Votes + 1), votes[['age_vot','Score', 'Comments','Ans_rank','Ans_count']], M=sm.robust.norms.HuberT())
hub_results = huber_t.fit()

#print hub_results.params
#print hub_results.bse
#varnames = ['var_%d' % i for i in range(len(hub_results.params))]
print hub_results.summary()

                    Robust linear Model Regression Results                    
Dep. Variable:                  Votes   No. Observations:                 2669
Model:                            RLM   Df Residuals:                     2664
Method:                          IRLS   Df Model:                            4
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Thu, 17 Mar 2016                                         
Time:                        04:50:12                                         
No. Iterations:                    25                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
age_vot        0.0285      0.001     23.031      0.0